In [ ]:
##### Weather Bench Work Attempt 3

In [1]:
import apache_beam

In [2]:
import os

In [3]:
import weatherbench2

In [4]:
import xarray as xr

In [5]:
import numpy as np
import math
from weatherbench2.regions import SliceRegion, ExtraTropicalRegion
from weatherbench2.evaluation import evaluate_in_memory
from weatherbench2 import config
from weatherbench2.metrics import MSE, ACC, Bias, MAE, SEEPS 

In [6]:
forecast_path = 'gs://weatherbench2/datasets/hres/2016-2022-0012-64x32_equiangular_conservative.zarr'
obs_path = 'gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr'
climatology_path = 'gs://weatherbench2/datasets/era5-hourly-climatology/1990-2019_6h_64x32_equiangular_conservative.zarr'


In [8]:
climatology = xr.open_zarr(climatology_path)

In [ ]:
paths1 = config.Paths(
    forecast=forecast_path,
    obs=obs_path,
    output_dir='./',   # Directory to save evaluation results
)

selection1 = config.Selection(
    variables=[
        'geopotential',
    ],
    levels=[500],
    time_slice=slice('2020-01-01', '2020-12-31'),
)

#https://weatherbench2.readthedocs.io/en/latest/init-vs-valid-time.html

#by init-time is set to true. (offical convention is init time true)
#
data_config1 = config.Data(selection=selection1, paths=paths1)

regions1 = {
    'global': SliceRegion(),
}

eval_configs1 = {
  'Deterministicfull': config.Eval(
      metrics={
          'mse': MSE(),
          'acc': ACC(climatology=climatology),
          'bias': Bias(),
          'mae': MAE(),
          #'seeps': SEEPS(climatology=climatology)
      },
      regions=regions1
  )
}


evaluate_in_memory(data_config1, eval_configs1)



In [10]:
results = xr.open_dataset('./Deterministicfull.nc')
#print(results)
# 0ACC, 1Bias, 2MAE, 3MSE
truemse = results['geopotential'][0,:,:,:].values
truemse

array([[[0.99958751],
        [0.99960745],
        [0.99919485],
        [0.99918863],
        [0.99861874],
        [0.99840055],
        [0.99747869],
        [0.99691364],
        [0.995603  ],
        [0.99458492],
        [0.99252413],
        [0.99075045],
        [0.98763458],
        [0.98469715],
        [0.98020307],
        [0.97575813],
        [0.96934508],
        [0.96298133],
        [0.95431291],
        [0.9454928 ],
        [0.93435492],
        [0.92304466],
        [0.9092302 ],
        [0.89519184],
        [0.87844481],
        [0.86153516],
        [0.84207021],
        [0.82257733],
        [0.80075497],
        [0.7793906 ],
        [0.75622167],
        [0.7334473 ],
        [0.70903192],
        [0.68533846],
        [0.66096998],
        [0.6376533 ],
        [0.61389917],
        [0.59097026],
        [0.56730423],
        [0.5444894 ],
        [0.5215291 ]]])

In [11]:
forecast = xr.open_zarr(forecast_path)
observations = xr.open_zarr(obs_path)

In [12]:
a1 =forecast['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-12-31'))[:,0,:,:]
b1 = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2020-12-31'))[::2,:,:]

errors = a1-b1
errors[:,:,:]

latitude = forecast['latitude'][:].values
latitude
delta = 2.8125
theta_upper = latitude + delta
theta_lower = latitude - delta

# Calculate weights based on the provided formula
weights = (np.sin(np.radians(theta_upper)) - np.sin(np.radians(theta_lower)))
weights /= weights.sum()
weights *= 32

#print(weights) #same as functions

weightedmatrix = (errors.values**2) * weights[None,None,:]

np.sum(weightedmatrix/(64*32*732))

524.1704827154813

In [13]:
#Wrong forecast method
a = forecast['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-12-31'))[:,:,:,:]
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2020-12-31'))[:,:,:]
adjustedb = b[::2,:,:]

weightedstuff = ((a[:-1,2,:,:].values-adjustedb[1:,:,:].values)**2) * weights[None,None,:]

np.sum(weightedstuff/(64*32*731))

1030.9059537729684

In [14]:
#Corrected forecast method
a = forecast['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-12-31'))[:,:,:,:]
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2021-01-01'))[:,:,:]
adjustedb = b[::2,:,:]
#adjustedb[1:-1,:,:]

weightedstuff = ((a[:,2,:,:].values-adjustedb[1:-1,:,:].values)**2) * weights[None,None,:]

np.sum(weightedstuff/(64*32*732))

1030.916475361522

In [15]:
#Corrected forecast method next 12 hour method of day incrementation works
a = forecast['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-12-31'))[:,:,:,:]
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2021-01-02'))[:,:,:]
adjustedb = b[::2,:,:]

weightedstuff = ((a[:,4,:,:].values-adjustedb[2:-2,:,:].values)**2) * weights[None,None,:]

np.sum(weightedstuff/(64*32*732))

1768.2778175289834

In [16]:
#6 Hour prediction?
a = forecast['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-12-31'))[:,:,:,:]
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2020-12-31'))[:,:,:]
adjustedb=b[1::2,:,:]

weightedstuff = ((a[:,1,:,:].values-adjustedb.values)**2) * weights[None,None,:]

np.sum(weightedstuff/(64*32*732))

497.4548952177726

In [17]:
#Full iterations MSE solved. 
values = []
a = forecast['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-12-31'))[:,:,:,:]
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2021-01-10'))[:,:,:].values

for i in range(41):
    if (i == 40):
        errors = a[:,i,:,:].values - b[i:,:,:][::2,:,:]
    else:
        errors = a[:,i,:,:].values - b[i:i-40,:,:][::2,:,:]
    
    weighted = (errors**2) * weights[None,None,:]
    val = np.sum(weighted/(64*32*732))
    values.append(val)

values

[524.1704827154813,
 497.4548952177726,
 1030.916475361522,
 1030.1687258289728,
 1768.2778175289834,
 2030.274565356739,
 3211.8475461081644,
 3902.784659040777,
 5585.526199027472,
 6833.673373690996,
 9457.873483175194,
 11643.57433446745,
 15598.756076276364,
 19229.11284268184,
 24930.02636700998,
 30431.139283286684,
 38550.10984532432,
 46417.25265759376,
 57373.65016703084,
 68269.29688948949,
 82332.716629519,
 96277.06321415906,
 113742.19098379147,
 131081.8548483217,
 152276.29657049198,
 173156.76709596973,
 197812.15040892668,
 221829.93103900785,
 249627.26615272494,
 276054.27621993166,
 305749.609196785,
 334062.5764581361,
 365618.7788792168,
 395114.04110002663,
 426826.5796435615,
 455806.3077243354,
 486726.0169611661,
 514979.56851804577,
 545780.1105445127,
 573745.5369142373,
 603819.2421187856]

In [18]:
#Full iterations MAE solved. 
values = []
a = forecast['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-12-31'))[:,:,:,:]
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2021-01-10'))[:,:,:].values

for i in range(41):
    if (i == 40):
        errors = a[:,i,:,:].values - b[i:,:,:][::2,:,:]
    else:
        errors = a[:,i,:,:].values - b[i:i-40,:,:][::2,:,:]
    
    weighted = abs(errors) * weights[None,None,:]
    val = np.sum(weighted/(64*32*732))
    values.append(val)

values

[17.812056334284577,
 17.164159959783543,
 24.94073795579501,
 24.332454799593904,
 31.658542099604134,
 33.046962398232765,
 41.30093180026586,
 44.184623976536066,
 52.22354534568563,
 55.72588159733174,
 65.35363988956514,
 70.24802821767139,
 81.24875325679595,
 87.64088552856049,
 99.67998750419287,
 107.3415802280933,
 121.17040259521676,
 130.06296454158763,
 145.0126381147597,
 155.54594551730068,
 171.3995643188193,
 182.9634454172762,
 199.77540231026202,
 212.2741946475557,
 229.84099475125856,
 243.1863174749243,
 261.08019154672724,
 275.2020800198121,
 293.24556727816224,
 307.5515271285009,
 324.9075592775503,
 339.1175336331174,
 356.1305955968538,
 369.9438605762256,
 385.828480970319,
 399.1314212189379,
 413.8759688638569,
 426.2183949539381,
 440.04890770822976,
 451.5128940678072,
 464.4714652167324]

In [19]:
#Full iterations Bias solved. 

values = []
a = forecast['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-12-31'))[:,:,:,:]
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2021-01-10'))[:,:,:].values

for i in range(41):
    if (i == 40):
        errors = a[:,i,:,:].values - b[i:,:,:][::2,:,:]
    else:
        errors = a[:,i,:,:].values - b[i:i-40,:,:][::2,:,:]
    
    weighted = errors * weights[None,None,:]
    val = np.sum(weighted/(64*32*732))
    values.append(val)

values

[-2.6391968671265142,
 0.012924315513474697,
 -3.0789752858098285,
 -0.5306739591431023,
 -3.9898913474765427,
 -1.1709010461211384,
 -4.404159698495218,
 -1.478389818174753,
 -4.6290611530716035,
 -1.6640611112849817,
 -5.000882057630287,
 -2.217105556276718,
 -5.637273331909876,
 -2.915341020183312,
 -6.469811932863462,
 -3.7616166186421154,
 -7.216519140303969,
 -4.605246038834386,
 -8.221979207204576,
 -5.635353802276816,
 -9.235218423775699,
 -6.777647101508959,
 -10.337436818163683,
 -7.72844180505211,
 -11.123269433549153,
 -8.537342209915305,
 -11.977552216121012,
 -9.300977456855884,
 -12.668885532549211,
 -10.05956079321408,
 -13.523694564300898,
 -10.83768334747731,
 -14.132490995660415,
 -11.438827420212561,
 -14.803313580481326,
 -12.076574006512255,
 -15.418214683873398,
 -12.710587779886673,
 -16.05875055292577,
 -13.237551072133398,
 -16.481026266972755]

In [20]:
#Full iterations ACC completed:
##2020 is conveniently a leap year, and climatology has 366 days

values = []
a = forecast['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-12-31'))[:,:,:,:]
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2021-01-10'))[:,:,:].values

newc= climatology['geopotential'].sel(level=500)
newc1 = newc.stack(time=("dayofyear", "hour")).transpose("time", "longitude", "latitude")
initial_days = newc1.isel(time=slice(0, 40))
newc1 = xr.concat([newc1, initial_days], dim="time").values


for i in range(41):
    if (i == 40):
        f = a[:,i,:,:].values
        o = b[i:,:,:][::2,:,:]
        ct = newc1[i:,:,:][::2,:,:]
    else:
        f= a[:,i,:,:].values
        o = b[i:i-40,:,:][::2,:,:]
        ct = newc1[i:i-40,:,:][::2,:,:]
    
    fprime = f - ct
    oprime = o - ct

    weightedtop = (fprime * oprime) * weights[None,None,:]
    top = weightedtop.sum(axis=(1,2))

    bottom1 = ((fprime**2) * weights[None,None,:]).sum(axis=(1,2))
    bottom2 = ((oprime**2) * weights[None,None,:]).sum(axis=(1,2))
    bottom = (bottom1*bottom2)**0.5

    val = np.sum(top/bottom)/732
  
    values.append(val)

values


[0.9995875081842586,
 0.9996074473923667,
 0.9991948457992847,
 0.999188625395613,
 0.9986187365597586,
 0.9984005461854758,
 0.9974786879044972,
 0.9969136420738255,
 0.995602998482821,
 0.9945849243007032,
 0.9925241276968452,
 0.9907504489598451,
 0.9876345796270348,
 0.9846971500872218,
 0.9802030703880322,
 0.9757581271311125,
 0.9693450777206665,
 0.9629813284091571,
 0.9543129125195475,
 0.9454928014811651,
 0.934354917602574,
 0.923044658775156,
 0.9092302014022656,
 0.8951918447385978,
 0.8784448135428513,
 0.8615351631528,
 0.8420702054707279,
 0.8225773330050634,
 0.800754971545485,
 0.7793906025058016,
 0.7562216746303004,
 0.73344729765995,
 0.709031923645106,
 0.6853384604831054,
 0.6609699772430093,
 0.6376532952145428,
 0.6138991717197894,
 0.590970262697181,
 0.5673042302212654,
 0.5444893972909391,
 0.5215290979870922]

In [120]:
newc= climatology['geopotential'].sel(level=500)
newc1 = newc.stack(time=("dayofyear", "hour")).transpose("time", "longitude", "latitude")
initial_days = newc1.isel(time=slice(0, 40))
newc1 = xr.concat([newc1, initial_days], dim="time")
newc1


<xarray.DataArray 'geopotential' (time: 1504, longitude: 64, latitude: 32)>
dask.array<concatenate, shape=(1504, 64, 32), dtype=float32, chunksize=(1464, 64, 32), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 -87.19 -81.56 -75.94 ... 75.94 81.56 87.19
    level      int64 500
  * longitude  (longitude) float64 0.0 5.625 11.25 16.88 ... 343.1 348.8 354.4
  * time       (time) object MultiIndex
  * dayofyear  (time) int64 1 1 1 1 2 2 2 2 3 3 3 ... 8 8 8 9 9 9 9 10 10 10 10
  * hour       (time) int64 0 6 12 18 0 6 12 18 0 6 ... 18 0 6 12 18 0 6 12 18
Attributes:
    long_name:      Geopotential
    short_name:     z
    standard_name:  geopotential
    units:          m**2 s**-2

In [114]:
b23 = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2021-01-10'))[:,:,:]
b23.shape

(1504, 64, 32)

In [101]:
adjustedb

<xarray.DataArray 'geopotential' (time: 732, longitude: 64, latitude: 32)>
dask.array<getitem, shape=(732, 64, 32), dtype=float32, chunksize=(50, 64, 32), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 -87.19 -81.56 -75.94 ... 75.94 81.56 87.19
    level      int64 500
  * longitude  (longitude) float64 0.0 5.625 11.25 16.88 ... 343.1 348.8 354.4
  * time       (time) datetime64[ns] 2020-01-01T06:00:00 ... 2020-12-31T18:00:00
Attributes:
    long_name:      Geopotential
    short_name:     z
    standard_name:  geopotential
    units:          m**2 s**-2

In [32]:
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2021-01-10'))[:,:,:]
b[0:-40,:,:][::2,:,:]
b[1:-39,:,:][::2,:,:]
b[2:-38,:,:][::2,:,:]

b[40:,:,:][::2,:,:]

<xarray.DataArray 'geopotential' (time: 732, longitude: 64, latitude: 32)>
dask.array<getitem, shape=(732, 64, 32), dtype=float32, chunksize=(50, 64, 32), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 -87.19 -81.56 -75.94 ... 75.94 81.56 87.19
    level      int64 500
  * longitude  (longitude) float64 0.0 5.625 11.25 16.88 ... 343.1 348.8 354.4
  * time       (time) datetime64[ns] 2020-01-11 ... 2021-01-10T12:00:00
Attributes:
    long_name:      Geopotential
    short_name:     z
    standard_name:  geopotential
    units:          m**2 s**-2